In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pydicom
!pip install efficientnet_pytorch
# -*- coding: utf-8 -*-
from torchvision import datasets, transforms, models
import torch
from torch import nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data.dataloader import default_collate
import numpy as np
import pydicom
import matplotlib.pyplot as plt
import glob
import os
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from PIL import Image
import ntpath
from efficientnet_pytorch import EfficientNet
import csv

     |████████████████████████████████| 1.9MB 17.5MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp36-none-any.whl size=16031 sha256=321e5c20fc5a4740144d41884edbad0947917da0f00b1a18978376771b9615db
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch


In [3]:
data_dir = "/content/drive/Shared drives/數位醫學/Case Presentation 2"

# hyper parameters
num_classes = 6
batch_size = 8
EPOCH = 10
#input_size = 256
input_size = 299
#input_size = 512

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# windowing
def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(dcm, window_center, window_width):
    
    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
      correct_dcm(dcm)
    
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)

    return img

def bsb_window(dcm):
    brain_img = window_image(dcm, 40, 80)
    subdural_img = window_image(dcm, 80, 200)
    #soft_img = window_image(dcm, 40, 380)
    bone_img = window_image(dcm , 600 , 2800)

    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img - (-20)) / 200
    #soft_img = (soft_img - (-150)) / 380
    bone_img = (bone_img - (-800)) / 2800

    #bsb_img = np.array([brain_img, subdural_img, soft_img]).transpose(1,2,0)
    bsb_img = np.array([brain_img, subdural_img, bone_img]).transpose(1,2,0)

    return bsb_img

In [5]:
# output: testData_img, testData_path
class myTestDataset(Dataset):
  def __init__(self, folder):
    #self.dictonary = {'epidural':0, 'healthy':1, 'intraparenchymal':2, 'intraventricular':3, 'subarachnoid':4, 'subdural':5}
    self.path = []
    for i in glob.glob(folder + '/*'):
      self.path.append(i)

  def __len__(self):
    return len(self.path)
 
  def __getitem__(self, idx):
    img = torch.from_numpy( bsb_window(pydicom.dcmread(self.path[idx])).transpose(2, 0, 1) ).type(torch.FloatTensor)
    return img, self.path[idx]

TestDataset = myTestDataset(folder='/content/drive/Shared drives/數位醫學/Case Presentation 2/TestingData')
batch_size = 1 # 懶得處理多tuple的狀況，直接改成1個1個來(別改)
test_loader = torch.utils.data.DataLoader(TestDataset, batch_size=batch_size, shuffle=True, num_workers=8)

In [6]:
'''
# 仍需要pretrained model的weight
# load pretrained weight(trained from ImageNet)
net = EfficientNet.from_pretrained('efficientnet-b4', num_classes=num_classes)
#print(net)
net = net.to(device)
'''

"\n# 仍需要pretrained model的weight\n# load pretrained weight(trained from ImageNet)\nnet = EfficientNet.from_pretrained('efficientnet-b4', num_classes=num_classes)\n#print(net)\nnet = net.to(device)\n"

In [7]:
# load pretrained weight(trained from ImageNet)
net = models.inception_v3(pretrained=True)
#set_parameter_requires_grad(net, feature_extract)
# Handle the auxilary net
num_ftrs = net.AuxLogits.fc.in_features
net.AuxLogits.fc = nn.Linear(num_ftrs, 6)
# Handle the primary net
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs,6)
#net.dropout = nn.Dropout(0.95)
net = net.to(device)
input_size = 299

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth


In [8]:
# 這行可換成要使用的weight
if device == torch.device('cpu'):
  model = torch.load('/content/drive/Shared drives/數位醫學/Case Presentation 2/Implementation/weights/b4(acc).pkl' ,map_location='cpu')
else:
  model = torch.load('/content/drive/Shared drives/數位醫學/Case Presentation 2/Implementation/weights/b4(acc).pkl')

# 用來存放{ dcm_path : predicted_label }
testToLabel = {}

# 將對應的號碼轉回文字
inv_dict = {0:'epidural', 1:'healthy', 2:'intraparenchymal', 3:'intraventricular', 4:'subarachnoid', 5:'subdural'}

# predicted img's label
with torch.no_grad():
  for data in test_loader:
    model.eval()
    images, path = data
    images = images.to(device)
    outputs = model(images)
    predicted_label = np.squeeze((torch.max(outputs.data, 1)[1]).cpu().numpy()).item()
    predicted_label = inv_dict[predicted_label]
    path = ''.join(path) # tuple => string
    filename = os.path.split(path)[1].replace('.dcm', '')
    testToLabel[filename] = predicted_label

In [9]:
testToLabel

{'Test_001': 'intraventricular',
 'Test_002': 'intraparenchymal',
 'Test_003': 'subdural',
 'Test_004': 'healthy',
 'Test_005': 'intraparenchymal',
 'Test_006': 'healthy',
 'Test_007': 'epidural',
 'Test_008': 'subdural',
 'Test_009': 'intraventricular',
 'Test_010': 'subarachnoid',
 'Test_011': 'epidural',
 'Test_012': 'healthy',
 'Test_013': 'intraparenchymal',
 'Test_014': 'subdural',
 'Test_015': 'subdural',
 'Test_016': 'epidural',
 'Test_017': 'subarachnoid',
 'Test_018': 'intraparenchymal',
 'Test_019': 'intraparenchymal',
 'Test_020': 'epidural',
 'Test_021': 'intraventricular',
 'Test_022': 'epidural',
 'Test_023': 'healthy',
 'Test_024': 'subdural',
 'Test_025': 'intraventricular',
 'Test_026': 'epidural',
 'Test_027': 'intraventricular',
 'Test_028': 'subarachnoid',
 'Test_029': 'epidural',
 'Test_030': 'subdural',
 'Test_031': 'subdural',
 'Test_032': 'subdural',
 'Test_033': 'healthy',
 'Test_034': 'subdural',
 'Test_035': 'intraventricular',
 'Test_036': 'subdural',
 'Tes

In [11]:
# 存成csv
# 跑出來csv檔沒東西的話再執行一次
w = csv.writer(open('/content/drive/Shared drives/數位醫學/Case Presentation 2/Implementation/testing_submission_trail0_CT_G09.csv', 'w'))
for data in [[k,testToLabel[k]] for k in sorted(testToLabel.keys())]:
  w.writerow(data)